# 1. Producing the streaming data <a class="anchor" id="1"></a>

In [ ]:
# import statements
from time import sleep
from kafka import KafkaProducer
from json import dumps
import datetime as dt
import csv

def read_csv(fileName):
    list = []
    with open(fileName, 'rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            list.append(row)
    return list

def connect_kafka_producer():
    producer = None
    try:
        producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return producer
    
def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

if __name__ == '__main__':
    
    topic = 'flightTopic'
    rows = []
    for counter in range(1,21):
        fileName = 'flight' + str(counter) + '.csv'
        rows += read_csv(fileName)
    
    print('Publishing records..')
    flightProducer = connect_kafka_producer()
    
    n = 10
    for i in range(0,len(rows),n):
        timestamp = int(dt.datetime.utcnow().timestamp())
        batch = rows[i:i + n]
        for item in batch:
            item['ts'] = timestamp
        publish_message(flightProducer, topic, batch)
        sleep(1)

Publishing records..
Message published successfully. Data: [{'YEAR': '2015', 'MONTH': '2', 'DAY': '6', 'DAY_OF_WEEK': '5', 'AIRLINE': 'OO', 'FLIGHT_NUMBER': '6271', 'TAIL_NUMBER': 'N937SW', 'ORIGIN_AIRPORT': 'FAR', 'DESTINATION_AIRPORT': 'DEN', 'SCHEDULED_DEPARTURE': '1712', 'DEPARTURE_TIME': '1701', 'DEPARTURE_DELAY': '-11', 'TAXI_OUT': '15', 'WHEELS_OFF': '1716', 'SCHEDULED_TIME': '123', 'ELAPSED_TIME': '117', 'AIR_TIME': '95', 'DISTANCE': '627', 'WHEELS_ON': '1751', 'TAXI_IN': '7', 'SCHEDULED_ARRIVAL': '1815', 'ARRIVAL_TIME': '1758', 'ARRIVAL_DELAY': '-17', 'DIVERTED': '0', 'CANCELLED': '0', 'CANCELLATION_REASON': '', 'AIR_SYSTEM_DELAY': '', 'SECURITY_DELAY': '', 'AIRLINE_DELAY': '', 'LATE_AIRCRAFT_DELAY': '', 'WEATHER_DELAY': '', 'ts': 1653607688}, {'YEAR': '2015', 'MONTH': '1', 'DAY': '19', 'DAY_OF_WEEK': '1', 'AIRLINE': 'AA', 'FLIGHT_NUMBER': '1605', 'TAIL_NUMBER': 'N496AA', 'ORIGIN_AIRPORT': 'DFW', 'DESTINATION_AIRPORT': 'ONT', 'SCHEDULED_DEPARTURE': '1740', 'DEPARTURE_TIME': '1